# Option Pricing under Normal Dynamics with Stochastic Volatility

 Link to paper: https://arxiv.org/pdf/1909.08047.pdf

Abstract: In this paper, we derive the price of a European call option of an asset following a normal process assuming stochastic volatility. The volatility is assumed to follow the Cox–Ingersoll– Ross (CIR) process. We then use the fast Fourier transform (FFT) to evaluate the option price given we know the characteristic function of the return analytically. We compare the results of fast Fourier transform with the Monte-Carlo simulation results of our process.

The diffusion equation for the price of an asset at time t:

 $$ dS(t) = rdt + \sqrt v(t)dW_1(t) $$ 

The volatility is assumed to be stochastic following the equation:

$$ dv = (a - bv) dt + \sigma \sqrt v(t) dW_2(t) $$



# Monte Carlo Simulation 

In [60]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft

In [61]:
#sample data

S = -0.001; #spot price
K = 0.000; #strike price
T = 1; #time of expiry
r = 0; #rate (similar to interest rate in Heston Model)
v = 0.09; #initial variance
theta = 0.0000005;  
kappa = 1;
sigma = 0.25;
rho = -0.9; 
x = 0;  

In [62]:
path = 100000;  #sample paths
iterations = 10;   

dt = 0.01
N = int(T/dt)
dt = T/N

In [63]:
#Absorption

S_A = np.zeros((path,N+1));
V_At = np.zeros((path,N+1));
V_A = np.zeros((path,N+1));
V_At[:,0] = v;
V_A[:,0] = v
S_A[:,0] = S

In [64]:
for i in range (iterations):
    
    Z1 = np.random.normal(size=(path,N))
    dW1 = np.sqrt(dt)*(Z1);
    Z2 = np.random.normal(size=(path,N))
    dW2 = rho*dW1+np.sqrt(1-rho**2)*Z2*np.sqrt(dt);
    
    for j in range(N):
        V_At[:,j+1] = V_At[:,j] + kappa*(theta - V_At[:,j])*dt + sigma*np.sqrt(np.maximum(V_At[:,j],0))*dW1[:,j];
        V_A[:,j+1] = np.maximum(V_At[:,j+1],0);
        S_A[:,j+1] = ((S_A[:,j])+ (r)*dt+ np.sqrt(V_A[:,j])*dW2[:,j]);
        payoff_fun = lambda x: np.maximum(x-K,0)
        payoff = payoff_fun(S_A[:, int(T*N)])
        C_A = (np.exp(-r*T)*(payoff))

In [65]:
payoff = np.fmax(S_A[:,-1]-K, 0)
mc_pricing = np.exp(-r*T) * np.mean(payoff)
print (mc_pricing)

0.09190546834879283


# Fast Fourier Transform

In [66]:
def chf_st(phi, t, r, q, S0, V0, theta, kappa, sigma, rho):
        d = np.sqrt((sigma ** 2) * (phi ** 2) + (kappa - 1j * rho * sigma * phi) ** 2)
        beta = kappa - 1j * phi * rho * sigma
        g = (beta - d) / (beta + d)
        D_t = (beta - d) / (sigma ** 2) * ((1 - np.exp(-d * t)) / (1 - g * np.exp(-d * t)))
        C_t = -1j * phi * (r - q) * t + kappa * theta / (sigma ** 2) * (
            (beta - d) * t - 2 * np.log((1 - g * np.exp(-d * t)) / (1 - g)))
        return np.exp(C_t + D_t * V0 + 1j * phi * S0)

In [67]:
def carr_madan_fft_call_pricer(N, eta, alpha, k_u, r, t, S0, V0, theta, kappa, sigma, rho, q, chf_st):

    def chf_st(phi, t, r, q, S0, V0, theta, kappa, sigma, rho):
        d = np.sqrt((sigma ** 2) * (phi ** 2) + (kappa - 1j * rho * sigma * phi) ** 2)
        beta = kappa - 1j * phi * rho * sigma
        g = (beta - d) / (beta + d)
        D_t = (beta - d) / (sigma ** 2) * ((1 - np.exp(-d * t)) / (1 - g * np.exp(-d * t)))
        C_t = -1j * phi * (r - q) * t + kappa * theta / (sigma ** 2) * (
            (beta - d) * t - 2 * np.log((1 - g * np.exp(-d * t)) / (1 - g)))
        return np.exp(C_t + D_t * V0 + 1j * phi * S0)
    

    lmda = 2 * np.pi / (N * eta)
    b = k_u - lmda * N / 2
    u_arr = np.arange(N) * eta
    
    
    k_arr = b + np.arange(N) * lmda
    delta_arr = np.zeros(N)
    delta_arr[0] = 1
    w_arr = (eta / 3) * (3 + (-1) ** (np.arange(N) + 1) - delta_arr)
    call_chf = (np.exp(-r * t) / ((alpha + 1j * u_arr) * (alpha + 1j * u_arr))) * (chf_st(u_arr - (alpha) * 1j,
                t, r, q, S0, V0, theta, kappa, sigma, rho))
    x_arr = np.exp(-1j * b * u_arr) * call_chf * w_arr
    
    fft_prices = (fft(x_arr))
    call_prices = (np.exp(-alpha * k_arr) / np.pi) * fft_prices.real
    
    
    u_w = ((k_u - b)* N * eta/(2*np.pi))+1
    x = int(round(u_w))
    
    return (call_prices[x])   

fft_pricing = carr_madan_fft_call_pricer(120000, 1, 30, 0.0005, 0, 1, -0.001, 0.09, 0.0000005, 1, 0.25, -0.9, 0,
                                  (chf_st(1, 1, 0, 0, -0.001, 0.09, 0.0000005, 1, 0.25, -0.9)) )

print (fft_pricing)

0.0916280726285714


In [68]:
print("The difference between MC and FFT:", mc_pricing - fft_pricing)

The difference between MC and FFT: 0.00027739572022143777
